# Chapter 12-05
## 파이썬과 기계학습

### 기계학습에 의한 수치 예측

In [ ]:
# 데이터 작성
# 예측에 이용하는 데이터 만들기
import numpy as np

# 난수 seed(초깃값)를 설정
np.random.seed(9)
# 0에서 1까지 100개의 수치를 생성, 난수를 혼합하기 전의 x
x_orig = np.linspace(0, 1, 100)

def f(x):
    # x에 대응하는 sin값을 돌려주는 함수
    return np.sin(2 * np.pi * x)

# 0에서 1까지 100개가 흩어져 있는 샘플 데이터 x를 생성
x = np.random.uniform(0, 1, size=100)[:, np.newaxis]
# x에 대응하는 sin값에 난수를 더하여 샘플 데이터(y)를 생성
y = f(x)+np.random.normal(scale=0.3, size=100)[:, np.newaxis]

In [ ]:
# 그래프 작성
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
# 학습용 데이터와 테스트용 데이터를 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.8)

# 원래 sin값과 샘플 데이터를 plot
plt.plot(x_orig, f(x_orig), ls=':')
plt.scatter(x_train, y_train)
plt.xlim((0, 1))

In [ ]:
# 모델의 그래프 표시
# 최소 자승법의 다항식 근사를 사용하여 데이터를 학습
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# 2x2의 그래프를 그리는 준비
fig, axs = plt.subplots(2, 2, figsize=(8, 5))

# 차수 0, 1, 3, 9에 대하여 학습한 결과를 표시
for ax, deg in zip(axs.ravel(), [0, 1, 3, 9]):
    # 파이프라인을 만듦
    e = make_pipeline(PolynomialFeatures(deg), LinearRegression())
    # 학습 set으로 학습
    e.fit(x_train, y_train)
    # 원래의 x로 예측
    px = e.predict(x_orig[:, np.newaxis])
    # 예측결과 그래프와 테스트 데이터 점을 그림
    ax.scatter(x_train, y_train)
    ax.plot(x_orig, px)
    ax.set(xlim=(0, 1), ylim=(-2, 2),
           ylabel='y', xlabel='x',
           title='degree={}'.format(deg))

plt.tight_layout()

In [ ]:
# 예측치와의 오차를 그래프화 한다
# 테스트 데이터를 사용하여 오버피팅의 모습을 그래프로 표시한다
from sklearn.metrics import mean_squared_error

# 실 데이터와의 오차를 보존하는 array
train_error = np.empty(10)
test_error = np.empty(10)
# 차수 0에서 9에 대하여 조사
for deg in range(10):
    # 모델을 작성
    e = make_pipeline(PolynomialFeatures(deg), LinearRegression())
    e.fit(x_train, y_train)
    # 테스트 데이터를 사용하여 예측값과 실제값의 오차를 조사
    train_error[deg] = mean_squared_error(y_train, e.predict(x_train))
    test_error[deg] = mean_squared_error(y_test, e.predict(x_test))

# 그래프를 그림
plt.plot(np.arange(10), train_error, ls=':', label='train')
plt.plot(np.arange(10), test_error, ls='-', label='test')
plt.ylim((0, 1))
plt.legend(loc='upper left')

### 이름으로 성별을 판정한다

In [ ]:
# array 작성
# 데이터를 읽어 들여 변환하기
import numpy as np
from sklearn.cross_validation import train_test_split

np.random.seed(9)
# 남녀의 태그가 붙어 있는 이름 데이터를 읽어 들임
txtbody = open('names.txt', encoding='utf-8')
# NumPy의 array로 변환
jnames = np.array([x.split() for x in txtbody], dtype='U12')
# 이름과 성별로 분할
names_train, gender_train, = jnames[:, 1], jnames[:, 0]

In [ ]:
# split_in_2words( ) 함수의 정의
# 히라가나 읽기를 2 문자마다 분할하는 함수를 만든다
def split_in_2words(name):
    # 이름을 2문자씩 분할하는 함수
    return [name[i:i+2] for i in range(len(name)-1)]

In [ ]:
# 함수 호출 테스트
split_in_2words("동방신기")

In [ ]:
# CountVectorizer 객체 작성
# 학습 데이터를 만들어 벡터화의 전단계로 데이터를 만들기
from sklearn.feature_extraction.text import CountVectorizer
bow_t = CountVectorizer(analyzer=split_in_2words).fit(names_train)

In [ ]:
# ‘かんかん(캉캉)’이라는 이름의 출현 수를 조사한다
name = 'かんかん'
b1 = bow_t.transform([name])
print(b1[0])

In [ ]:
# 문자열의 역 검색
# 출력 데이터에서 문자열을 역방향 조회
# 직전의 출력 셀에 표시된 ID로 바꿉니다
print(bow_t.get_feature_names()[283])
print(bow_t.get_feature_names()[1898])

In [ ]:
# 문자열의 출현 수를 조사한다
# 학습 데이터를 사용하여 문자열의 발생 횟수를 센다
names_bow = bow_t.transform(names_train)

In [ ]:
# TfidfTransformer 객체 생성
# TF-IDF에서 데이터의 가중치와 정규화를 실시
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_t = TfidfTransformer().fit(names_bow)

In [ ]:
# 가중치 표현 실행
# tfidf_t에서 어떤 변환이 수행되는지 확인
tfidf1 = tfidf_t.transform(b1)
print(tfidf1)

In [ ]:
# 학습 실행
# 나이브 베이즈의 다항 모델을 이용한 학습 분석기를 만듦
from sklearn.naive_bayes import MultinomialNB
# 문자열에 가중치를 붙이고 정규화를 실행
names_tfidf = tfidf_t.transform(names_bow)
# 학습실행
namegender_detector = MultinomialNB().fit(names_tfidf, gender_train)

In [ ]:
# 성별 판정
# 'かんかん(캉캉)'이라는 이름의 성별을 판정
print(namegender_detector.predict(tfidf1)[0])

In [ ]:
# predict_gender( ) 함수의 정의
# 문자열을 주고 성별을 예측하는 함수를 정의
def predict_gender(name):
    # 성별 판정
    bow = bow_t.transform([name])
    n_tfidf = tfidf_t.transform(bow)
    return namegender_detector.predict(n_tfidf)[0]

In [ ]:
# 함수 실행
print(predict_gender("のんな"))

In [ ]:
# 데이터를 학습 데이터와 테스트 데이터로 분할
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=split_in_2words)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [ ]:
# 파이프 라인을 사용하여 학습 및 테스트
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
scores = cross_val_score(pipeline,  # steps to convert raw messages into models
                         names_train,  # training data
                         gender_train,  # training labels
                         cv=10,  # split data randomly into 10 parts: 9 for training, 1 for scoring
                         scoring='accuracy',  # which scoring metric?
                         n_jobs=-1,  # -1 = use all cores = faster
                         )
print(scores)